# Registration
## Abstract
- This section describe registration algorithm for point cloud registration.

## Introduction


In [1]:
%load_ext autoreload
%autoreload 2

## Mathcing with Iterative Closest Point (ICP)

This subsection demonstrates registration with ICP for a object point cloud. The subsection tutorial use following packages:

In [2]:
import numpy as np

from tutlibs.matching import icp
from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.transformation import rotation, translation
from tutlibs.transformation import TransformationMatrix as tm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


The tutorial code is following:

In [3]:
# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = rotation(translation(target_xyz, np.array([0.0, 0.1, 0.0])), 'z', angle=30/180*np.pi)

# # keypoints estimation
source_v_xyz = voxel_grid_sampling(source_xyz, 0.01)
target_v_xyz = voxel_grid_sampling(target_xyz, 0.01)

# use ICP
trans_mat = icp(source_v_xyz, target_v_xyz, 20)
estimated_source_xyz = tm.transformation_Nx3_with_4x4(source_xyz, trans_mat)

# visualize results
source_colors = np.tile([[1, 0, 0]], (source_xyz.shape[0], 1))
estimated_source_colors = np.tile([[0, 1, 0]], (estimated_source_xyz.shape[0], 1))
target_colors = np.tile([[1, 1, 0]], (target_xyz.shape[0], 1))

vis_xyz = np.concatenate([estimated_source_xyz, target_xyz, source_xyz])
vis_color = np.concatenate([estimated_source_colors, target_colors, source_colors])
visualizer.k3d(vis_xyz, colors=vis_color)
# io.write('outputs/r1.ply', vis_xyz, vis_color)


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

Output()

## Use RANSAC

This subsection demonstrates regustration with ICP for a

## Use of Open3D

In [4]:
import numpy as np
import open3d as o3d

from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer
from tutlibs.sampling import voxel_grid_sampling
from tutlibs.transformation import rotation, translation
from tutlibs.transformation import TransformationMatrix as tm

In [8]:
def icp_o3d(source_xyz, target_xyz, threshold=0.1):
    source = o3d.geometry.PointCloud()
    target = o3d.geometry.PointCloud()

    source.points = o3d.utility.Vector3dVector(source_xyz)
    target.points = o3d.utility.Vector3dVector(target_xyz)

    trans_init = np.identity(4)
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))

    transformation = reg_p2p.transformation

    return transformation

# data acquisition
target_xyz, _, _ = io.read('../data/bunny_pc.ply')

# source random transformation
source_xyz = rotation(translation(target_xyz, np.array([0.0, 0.1, 0.0])), 'z', angle=30/180*np.pi)

# # keypoints estimation
source_v_xyz = voxel_grid_sampling(source_xyz, 0.01)
target_v_xyz = voxel_grid_sampling(target_xyz, 0.01)

# use ICP
trans_mat = icp_o3d(source_v_xyz, target_v_xyz)
estimated_source_xyz = tm.transformation_Nx3_with_4x4(source_xyz, trans_mat)

# visualize results
source_colors = np.tile([[1, 0, 0]], (source_xyz.shape[0], 1))
estimated_source_colors = np.tile([[0, 1, 0]], (estimated_source_xyz.shape[0], 1))
target_colors = np.tile([[1, 1, 0]], (target_xyz.shape[0], 1))

vis_xyz = np.concatenate([estimated_source_xyz, target_xyz, ])
vis_color = np.concatenate([estimated_source_colors, target_colors, ])
# visualizer.k3d(vis_xyz, colors=vis_color)
io.write('outputs/r1.ply', vis_xyz, vis_color)
